# The Journey

This section describes the decisions taken to get to our final solution
 

## The Baseline

This section is the code we were given

In [1]:
# requires sciket-learn 0.18
# if required, conda update scikit-learn

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.grid_search import GridSearchCV   #Perforing grid search


def readFiles():
    #Reading files
    X = pd.read_csv("trainingData.txt",sep='\t',header=None)
    Y = pd.read_csv("trainingTruth.txt",sep='\t',header=None)
    Y = np.array(Y).ravel()
    
    return (X,Y)


def preprocessData1( X, Y ):
    print('Preprocessing data.')
    
    # Replace NAs with 0
    X = X.fillna(0) 
    
    return (X,Y)


def runModel1(X,Y):
    print('Run model')
    
    clf = OneVsRestClassifier(RandomForestClassifier(n_estimators = 10,random_state=25))
    clf.fit(X,Y)
    Y_predict = clf.predict_proba(X)
    
    return Y_predict, clf


def calculateROC(Y, Y_predict):
    print('Calc ROC')
    # Binarize the output
    y_bin = label_binarize(Y, classes=[1, 2, 3,4])

    #Calculate AUC
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(4):
        fpr[i], tpr[i], _ = roc_curve(y_bin[:, i], Y_predict[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    return roc_auc

def createSubmission(clf):
    #Create submission
    Xtest = pd.read_csv("testData.txt",sep="\t",header=None)
    y_final_prob = clf.predict_proba(Xtest)
    y_final_label = clf.predict(Xtest)

    sample = pd.DataFrame(np.hstack([y_final_prob.round(5),y_final_label.reshape(y_final_prob.shape[0],1)]))
    sample.columns = ["prob1","prob2","prob3","prob4","label"]
    sample.label = sample.label.astype(int)
    
    #Submit this file to dropbox
    sample.to_csv("Johnston_Memic.csv",sep="\t" ,index=False,header=None)
    

# Read the files in.   
(X,Y) = readFiles()

# Clean up the data
(X,Y) = preprocessData1(X,Y)

# Run the model
Y_predict, clf = runModel1(X,Y)

baselineAUC = calculateROC(Y, Y_predict)

print('Baseline AUC is: ', baselineAUC)

#createSubmission(clf)




/Users/rob/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/rob/anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Preprocessing data.
Run model
Calc ROC
Baseline AUC is:  {0: 0.99996212682662722, 1: 0.99993172956870668, 2: 0.99996736488783788, 3: 0.99986086891635195}


## Pre-processing the data



### Analysing NAs

First we analyse the prevalance on NAs in our features. For each feature, we calculate the frequency of NAs. This will help determine whether we think the feature is worth keeping as too many NAs indicates that the feature is not giving us enough information.


In [2]:
# Read the files in.   
(X,Y) = readFiles()

sums = {}

total = X.shape[0]

for col in X.columns:
    # Count the NAs
    sums[col] = total - X[col].count()
    
# Do something more clever here? Plot a distribution?
#print(sums)
print('Maximum number of NAs in one column: ', max(sums.values()))
print('Minimum number of NAs in one column: ', min(sums.values()))

# Check the rows to see if there are any rows with excessive NAs
rowSums = X.isnull().sum(axis=1).tolist()
print('Maximum number of NAs in one row: ', max(rowSums))
print('Minimum number of NAs in one row: ', min(rowSums))


Maximum number of NAs in one column:  81
Minimum number of NAs in one column:  40
Maximum number of NAs in one row:  7
Minimum number of NAs in one row:  0


### NAs Replaced with Mean
We first looked at the treatment of nulls. The first step is, instead of replacing NAs with 0, replace them with the mean of the feature.

In [3]:


def preprocessData3( X, Y ):
    print('Preprocessing data.')

    # Q: Normalise data for SVMs - what about decision trees?
    

    #Rewrite this from the HW ipython book
    # Replace any NaN in X with the mean of the column
    # Replacing with the mean gives a better score
    xMean = []
    for col in X.columns:
        xMean = X[col].mean()
        #print(col, ' ', xMean)
        X.loc[X[col].isnull(), col] = xMean
    
    return (X,Y)


# Read the files in.   
(X,Y) = readFiles()


# Clean up the data
(X,Y) = preprocessData3(X,Y)

# Run the model
Y_predict, clf = runModel1(X,Y)

meanNAAUC = calculateROC(Y, Y_predict)

print('Baseline AUC is: ', baselineAUC)
print('Mean NA AUC is: ', meanNAAUC)




Preprocessing data.
Run model
Calc ROC
Baseline AUC is:  {0: 0.99996212682662722, 1: 0.99993172956870668, 2: 0.99996736488783788, 3: 0.99986086891635195}
Mean NA AUC is:  {0: 0.99998479648620131, 1: 0.99993036758884135, 2: 0.99994417906316257, 3: 0.99988691290227649}


## First Submission

I tried this this morning to submit a trial entry but it just kept on running.......

In [7]:
# This takes 86 min to run. Set to True if you want to run it, otherwise
# the results will be taken from a previous run
runLongRunningTest = False

def runModelWedSubmission(X,Y):
    print('Run model')
    
    model_to_set = OneVsRestClassifier(RandomForestClassifier())

    param_test1 = {'estimator__n_estimators':[10,20,30,40,50], 'estimator__max_depth':[3,6,8,12,24,32], 
               'estimator__min_samples_split':[2,4,6],'estimator__min_samples_leaf':[1,2,4]}
    gsearch1 = GridSearchCV(estimator = model_to_set, 
                        param_grid = param_test1,n_jobs=8,iid=False, cv=5,verbose=2)
    gsearch1.fit(X,Y)

    Y_predict = gsearch1.predict_proba(X)
    
    #clf = OneVsRestClassifier(RandomForestClassifier(n_estimators = 10,random_state=25))
    #clf.fit(X,Y)
    #Y_predict = clf.predict_proba(X)
    
    return Y_predict, clf


# Read the files in.   
(X,Y) = readFiles()


# Clean up the data
(X,Y) = preprocessData3(X,Y)



if runLongRunningTest:
    # Run the model
    Y_predict, clf = runModelWedSubmission(X,Y)

    wedSubmissionAAUC = calculateROC(Y, Y_predict)
    createSubmission(clf)
else:
    # Reults from previous run:
    #  [Parallel(n_jobs=8)]: Done 1350 out of 1350 | elapsed: 89.6min finished
    wedSubmissionAAUC =  {0: 0.99999891511100747, 1: 1.0, 2: 0.99999372674492126, 3: 1.0}


print('Baseline AUC is: ', baselineAUC)
print('Tuesday AUC is: ', wedSubmissionAAUC)



Preprocessing data.
Baseline AUC is:  {0: 0.99996212682662722, 1: 0.99993172956870668, 2: 0.99996736488783788, 3: 0.99986086891635195}
Tuesday AUC is:  {0: 0.9999989151110075, 1: 1.0, 2: 0.9999937267449213, 3: 1.0}



# The Final Solution

TBD

NB: When you run the sample (before upgrading) these are the values it produces: {0: 0.95755745500532141, 1: 0.94345356758244103, 2: 0.95754489510952012, 3: 0.935902875654121}